In [ ]:
import numpy as np
import pandas as pd
# Для сравнения может потребоваться sklearn, если разрешен для этой цели
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error

# Вспомогательный класс: Простое дерево регрессии (упрощенная версия)
class SimpleRegressionTree:
    """
    Очень простое дерево регрессии.
    Разделяет данные один раз по лучшему признаку и порогу,
    предсказывает среднее значение целевой переменной в каждом листе.
    """
    def __init__(self, max_depth=1):
        """
        Инициализация дерева.

        Args:
            max_depth (int): Максимальная глубина дерева. Для простого дерева регрессии
                             в GB обычно используют маленькую глубину (например, 1 или 3).
                             По умолчанию 1 (решающий пень).
        """
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left_value = None  # Предсказанное значение в левом листе
        self.right_value = None # Предсказанное значение в правом листе
        self.left_tree = None   # Левое поддерево (для max_depth > 1)
        self.right_tree = None  # Правое поддерево (для max_depth > 1)
        self.is_leaf = True     # Флаг, является ли узел листом
        self.prediction = None  # Предсказание для узла, если он лист

    def fit(self, X, y, depth):
        """
        Обучает дерево регрессии.

        Args:
            X (np.ndarray): Входные признаки.
            y (np.ndarray): Целевая переменная (остатки в контексте GB).
            depth (int): Текущая глубина дерева.
        """
        n_samples, n_features = X.shape

        # Если достигнута максимальная глубина или недостаточно примеров, делаем узел листом
        if depth >= self.max_depth or n_samples < 2:
            self.is_leaf = True
            self.prediction = np.mean(y) if n_samples > 0 else 0.0
            return

        # Находим лучшее разделение (признак и порог)
        best_split = self._find_best_split(X, y)

        if best_split is None:
             # Не удалось найти хорошее разделение, делаем узел листом
             self.is_leaf = True
             self.prediction = np.mean(y) if n_samples > 0 else 0.0
             return

        self.is_leaf = False
        self.feature_index = best_split['feature_index']
        self.threshold = best_split['threshold']

        # Разделяем данные
        left_indices = X[:, self.feature_index] <= self.threshold
        right_indices = ~left_indices # Инвертируем булевы индексы

        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        # Рекурсивно строим поддеревья
        self.left_tree = SimpleRegressionTree(max_depth=self.max_depth)
        self.left_tree.fit(X_left, y_left, depth + 1)

        self.right_tree = SimpleRegressionTree(max_depth=self.max_depth)
        self.right_tree.fit(X_right, y_right, depth + 1)


    def _find_best_split(self, X, y):
        """
        Находит лучшее разделение (признак и порог), минимизирующее MSE.
        """
        n_samples, n_features = X.shape
        best_mse = float('inf')
        best_split = None

        # Перебираем все признаки
        for feature_index in range(n_features):
            feature_values = X[:, feature_index]
            unique_values = np.unique(feature_values)

            # Перебираем потенциальные пороги
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2 if len(unique_values) > 1 else unique_values

            for threshold in thresholds:
                # Разделяем данные по текущему порогу
                left_indices = feature_values <= threshold
                right_indices = ~left_indices

                y_left, y_right = y[left_indices], y[right_indices]

                # Вычисляем MSE для текущего разделения
                # MSE = sum((y_i - mean_left)^2) + sum((y_j - mean_right)^2)
                mse = 0.0
                if len(y_left) > 0:
                    mean_left = np.mean(y_left)
                    mse += np.sum((y_left - mean_left)**2)
                if len(y_right) > 0:
                    mean_right = np.mean(y_right)
                    mse += np.sum((y_right - mean_right)**2)

                # Если текущее MSE меньше лучшего, обновляем
                if mse < best_mse:
                    best_mse = mse
                    best_split = {
                        'feature_index': feature_index,
                        'threshold': threshold
                    }

        return best_split


    def predict_single(self, x):
        """
        Прогнозирует значение для одного примера.
        """
        if self.is_leaf:
            return self.prediction

        if x[self.feature_index] <= self.threshold:
            if self.left_tree is not None:
                 return self.left_tree.predict_single(x)
            else:
                 # Этого не должно происходить при правильной постройке дерева
                 return self.prediction # Возвращаем предсказание родительского узла
        else:
            if self.right_tree is not None:
                 return self.right_tree.predict_single(x)
            else:
                 # Этого не должно происходить при правильной постройке дерева
                 return self.prediction # Возвращаем предсказание родительского узла


    def predict(self, X):
        """
        Прогнозирует значения для массива примеров.
        """
        X = np.asarray(X)
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples)
        for i in range(n_samples):
            predictions[i] = self.predict_single(X[i])
        return predictions


# Реализация алгоритма Gradient Boosting Regressor
class SimpleGradientBoostingRegressor:
    """
    Простая реализация Gradient Boosting Regressor
    с использованием простых деревьев регрессии в качестве базовых учеников.
    Использует только numpy и pandas.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        """
        Инициализация Gradient Boosting Regressor.

        Args:
            n_estimators (int): Количество базовых учеников (деревьев). По умолчанию 100.
            learning_rate (float): Скорость обучения (шаг градиентного спуска). По умолчанию 0.1.
            max_depth (int): Максимальная глубина базовых деревьев. По умолчанию 3.
        """
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.estimators = [] # Список базовых учеников (SimpleRegressionTree)
        self.initial_prediction = None # Начальное предсказание (среднее значение y)

    def fit(self, X, y):
        """
        Обучает модель Gradient Boosting.

        Args:
            X (np.ndarray или pd.DataFrame): Входные признаки.
            y (np.ndarray или pd.Series): Целевая переменная.
        """
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples = X.shape[0]

        # Шаг 1: Инициализируем модель константой (средним значением y)
        self.initial_prediction = np.mean(y)
        # Инициализируем текущие предсказания
        current_predictions = np.full(n_samples, self.initial_prediction)

        self.estimators = []

        # Основной цикл Gradient Boosting
        for _ in range(self.n_estimators):
            # Шаг 2: Вычисляем "отрицательные градиенты" (остатки)
            # Для MSE функции потерь, отрицательный градиент = (y - current_prediction)
            residuals = y - current_predictions

            # Шаг 3: Обучаем новое базовое дерево на остатках
            tree = SimpleRegressionTree(max_depth=self.max_depth)
            tree.fit(X, residuals, depth=0)

            # Шаг 4: Предсказываем остатки с помощью нового дерева
            tree_predictions = tree.predict(X)

            # Шаг 5: Обновляем текущие предсказания с учетом скорости обучения
            current_predictions += self.learning_rate * tree_predictions

            # Сохраняем обученное дерево
            self.estimators.append(tree)

    def predict(self, X):
        """
        Прогнозирует значения для входных данных.

        Args:
            X (np.ndarray или pd.DataFrame): Входные данные.

        Returns:
            np.ndarray: Спрогнозированные значения.
        """
        X = np.asarray(X)
        n_samples = X.shape[0]

        # Начинаем с начального предсказания
        predictions = np.full(n_samples, self.initial_prediction)

        # Добавляем предсказания от каждого базового дерева с учетом скорости обучения
        for tree in self.estimators:
            predictions += self.learning_rate * tree.predict(X)

        return predictions

# 2. Симуляция данных для тестирования Gradient Boosting Regressor

# Генерируем данные с нелинейной зависимостью и шумом
np.random.seed(42)
n_samples_sim = 200
X_sim = np.random.rand(n_samples_sim, 2) * 10 # Признаки от 0 до 10
# Целевая переменная: нелинейная функция признаков + шум
y_sim = 2 * X_sim[:, 0]**2 - 3 * X_sim[:, 1] + np.sin(X_sim[:, 0] * X_sim[:, 1]) + np.random.normal(0, 5, n_samples_sim)


# 3. Тестирование и сравнение

# Используем нашу реализацию Gradient Boosting Regressor
my_gbr = SimpleGradientBoostingRegressor(n_estimators=50, learning_rate=0.2, max_depth=2) # Уменьшим n_estimators для скорости
my_gbr.fit(X_sim, y_sim)
my_predictions = my_gbr.predict(X_sim)

# Вычисляем среднеквадратичную ошибку (MSE) на обучающих данных
my_mse = np.mean((y_sim - my_predictions)**2)
print(f"MSE нашей реализации Gradient Boosting Regressor на симулированных данных: {my_mse:.4f}")

# Сравнение со стандартной реализацией (sklearn)
# Для сравнения разрешено использовать sklearn
# try:
#     from sklearn.ensemble import GradientBoostingRegressor
#     from sklearn.metrics import mean_squared_error
#
#     # Используем GradientBoostingRegressor из sklearn
#     # random_state для воспроизводимости
#     sklearn_gbr = GradientBoostingRegressor(n_estimators=50, learning_rate=0.2, max_depth=2, random_state=42)
#     sklearn_gbr.fit(X_sim, y_sim)
#     sklearn_predictions = sklearn_gbr.predict(X_sim)
#
#     # Вычисляем MSE sklearn
#     sklearn_mse = mean_squared_error(y_sim, sklearn_predictions)
#     print(f"MSE sklearn GradientBoostingRegressor на симулированных данных: {sklearn_mse:.4f}")
#
#     # Сравнение предсказаний
#     predictions_mse = np.mean((my_predictions - sklearn_predictions)**2)
#     print(f"MSE между предсказаниями нашей и sklearn реализаций: {predictions_mse:.6f}")
#
# except ImportError:
#     print("\nБиблиотека sklearn не найдена. Пропуск сравнения.")
# except Exception as e:
#     print(f"\nОшибка при сравнении со sklearn: {e}")

# Визуализация (опционально, требует matplotlib)
# import matplotlib.pyplot as plt
#
# # Визуализация предсказаний vs истинных значений для одного признака (если признаков > 1, выбираем первый)
# if X_sim.shape[1] >= 1:
#     plt.figure(figsize=(10, 6))
#     # Сортируем по первому признаку для лучшей визуализации
#     sort_indices = np.argsort(X_sim[:, 0])
#     plt.scatter(X_sim[sort_indices, 0], y_sim[sort_indices], label='Истинные значения', alpha=0.6)
#     plt.plot(X_sim[sort_indices, 0], my_predictions[sort_indices], color='red', label='Наши предсказания', linewidth=2)
#     # if 'sklearn_predictions' in locals():
#     #      plt.plot(X_sim[sort_indices, 0], sklearn_predictions[sort_indices], color='green', linestyle='--', label='sklearn предсказания', linewidth=2)
#
#     plt.title('Сравнение предсказаний Gradient Boosting Regressor')
#     plt.xlabel('Признак 1')
#     plt.ylabel('Целевая переменная')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

